# Part 2: Email Behaviour Data Analysis

---

### Install Python packages (pip only)

In [64]:
#e.g., %pip install some-package
%pip install networkx
%pip install matplotlib
%pip install numpy
%pip install json

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


### Import Python packages

In [65]:
#e.g., import some-package
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json

---

### Task 1 of 1 

Examine the file "emails_cmt224.edgelist" which represents email behaviour at an organisation. Each line contains two numbers, 𝑢 and 𝑣, separated by a blank space. Consider each number as an identifier for an individual in an organisation, with the space on each line representing that the individual, 𝑢, sent at least one email to the another individual, 𝑣, at some point. Model the data using an appropriate, directed network representation and answer the following questions:

##### Q1. Do the majority of individuals have a higher or lower ratio of mutual connections than average in the network?

In [66]:
#CODE:
# Load data.
file_path = "emails_cmt224.edgelist"
eg = nx.read_edgelist(file_path, create_using=nx.DiGraph())
# Computing the reciprocity of each node.
reciprocity_per_node = nx.reciprocity(eg, eg.nodes())
# Creat a list.
reciprocity_values = list(reciprocity_per_node.values())
# Calculate the average reciprocity of all nodes.
mean_reciprocity = np.mean(reciprocity_values)
print(f"Mean reciprocity: {mean_reciprocity:.2f}")
# Compare the reciprocity of all nodes with the average, and count the number of values less than the average, the number of values greater than the average.
higher_count = 0
lower_count = 0
total_num = eg.number_of_nodes()
for node in reciprocity_values:
    if node > mean_reciprocity: higher_count += 1
    elif node < mean_reciprocity: lower_count += 1
# Calculate of the ratio of both.
higher_rate = higher_count / total_num
lower_rate = lower_count / total_num
print(f"The number of nodes' reciprocity higher than the average is {higher_count}, the ratio is: {higher_rate:.2f}")
print(f"The number of nodes' reciprocity lower than the average is {lower_count}, the ratio is: {lower_rate:.2f}")

Mean reciprocity: 0.52
The number of nodes' reciprocity higher than the average is 462, the ratio is: 0.65
The number of nodes' reciprocity lower than the average is 244, the ratio is: 0.35


##### Q2. Using the largest, strongly connected component (where at least one path exists between each individual and all others). Could the connectivity of the component be suggested to be reflective of a small world phenomenon in comparison to the typical connectivity of 10 comparative random networks?

In [67]:
#CODE:
# Find the most strongly connected component.
strongly_connected_components = nx.strongly_connected_components(eg)
# Sort by number of nodes.
strongly_connected_components_sorted_by_number_of_nodes = sorted(
    strongly_connected_components, 
    key=len,
    reverse=True
)
# Take the strongly connected component with the most nodes.
list_of_nodes_in_largest_strongly_connected_component = strongly_connected_components_sorted_by_number_of_nodes[0]
# Creating subgraphs.
LSCC = eg.subgraph(list_of_nodes_in_largest_strongly_connected_component).copy()
# Calculate the shortest path length and the average clustering coefficient of the subgraphs.
eg_sub_average_shortest_path_length = nx.average_shortest_path_length(LSCC)
eg_sub_average_clustering = nx.average_clustering(LSCC)
print(f"Average shortest path length in eg's largest strongly connected component: {eg_sub_average_shortest_path_length:.2f}")
print(f"Average clustering coefficient in eg's largest strongly connected component: {eg_sub_average_clustering:.2f}")

# Generate 10 random networks with the number of nodes and edges equal to the subgraphs.
random_average_shortest_path_length = []
random_average_clustering = []
for i in range(10):
    # Take the largest strongly connected component and generate a subgraph.
    R = nx.gnm_random_graph(LSCC.number_of_nodes(), LSCC.number_of_edges(), directed=True)
    R_LSCC_nodes = sorted(nx.strongly_connected_components(R), key=len, reverse=True)[0]
    R_LSCC = R.subgraph(R_LSCC_nodes).copy()
    # Calculate the shortest path length and the average clustering coefficient.
    random_average_shortest_path_length.append(nx.average_shortest_path_length(R_LSCC))
    random_average_clustering.append(nx.average_clustering(R_LSCC))
    
# Calculate the average of 10 random network results.
mean_random_average_shortest_path_length = np.mean(random_average_shortest_path_length)
mean_random_average_clustering = np.mean(random_average_clustering)
print(f"The mean of average shortest path length in 10 random world: {mean_random_average_shortest_path_length:.2f}")
print(f"The mean of average clustering in 10 random world: {mean_random_average_clustering:.2f}")

Average shortest path length in eg's largest strongly connected component: 2.56
Average clustering coefficient in eg's largest strongly connected component: 0.39
The mean of average shortest path length in 10 random world: 2.32
The mean of average clustering in 10 random world: 0.04


##### Q3. Are occurrences of induced, connected subgraphs of 3 individuals (triads) with only mutual connections more abundant in the largest, strongly connected component than those with a mixture of asymmetric and mutual connections? What does this suggest about how mutual connections are distributed in the component?

In [68]:
#CODE:
def calculate_normalised_connected_triadic_census(H):
    # Find all triples and return a dictionary.
    tc = nx.triadic_census(H)
    tc_rate = {}
    # Delete non-connected triples.
    del tc["003"]
    del tc["012"]
    del tc["102"]
    # model the data so that it sums to 1, in order to see the ratio.
    factor = 1.0 / sum(tc.values())
    for k in tc:
        tc_rate[k] = round(tc[k] * factor, 2)
    return tc, tc_rate

def triads_compared(tc, tc_rate):
    # Calculate the number of triples containing only mutually connections ("300").
    only_mutual_rate = tc_rate.get("300")
    only_mutual_num = tc.get("300")
    # Calculate the number of triples containing mixed connections.
    mixture_rate = sum(value for key, value in tc_rate.items() if key != "300")
    mixture_num = sum(value for key, value in tc.items() if key != "300")
    print(f"The rate of only mutual: {only_mutual_rate:2}")
    print(f"The number of only mutual: {only_mutual_num:2}")
    print(f"The rate of those with a mixture of asymmetric and mutual connections: {round(mixture_rate, 2)}")
    print(f"The number of those with a mixture of asymmetric and mutual connections: {round(mixture_num, 2)}")

# Run the function to compute the triple census of LSCC.
triadic_census, triadic_census_rate = calculate_normalised_connected_triadic_census(LSCC)
# Output triple census results.
print(triadic_census)
# Compare results.
triads_compared(triadic_census, triadic_census_rate)

{'021D': 31641, '021U': 15065, '021C': 25088, '111D': 62379, '111U': 96113, '030T': 2609, '030C': 292, '201': 107247, '120D': 3549, '120U': 4096, '120C': 3954, '210': 17333, '300': 12982}
The rate of only mutual: 0.03
The number of only mutual: 12982
The rate of those with a mixture of asymmetric and mutual connections: 0.97
The number of those with a mixture of asymmetric and mutual connections: 369366


---
### Task 2 of 2

Examine the JSON file "emails_cmt224_departments.json" (departments file). Keys in the departments file represent individuals using the same ids as in the "emails_cmt224.edgelist" file in Part 2, Task 1 and the values represent a department id that the individual can be attributed to. Using the contents of the departments file in combination with the network in Part 2, Task 1, answer the following questions:

##### Q1. Using the connections that individuals have in the network, are they more likely to mix with others in their department or those with a similar number of outward connections?

In [69]:
#CODE:
# Load the data
with open("emails_cmt224_departments.json") as json_file:
    departments = json.load(json_file)
# Data initialisation
department_connections = 0
similar_connections = 0

# Calculate the number of connections (degrees).
degrees = dict(eg.degree())
# Traverse each node.
for node in eg.nodes():
    this_department = departments[node]
    this_degee = degrees[node]
    # Analyse whether its connected nodes belong to the same sector or have a similar number of connections.
    for other in eg.neighbors(node):
        if departments[other] == this_department: department_connections += 1
        # Similarity is taken as 1.
        if abs(degrees[other] - this_degee) == 1: similar_connections += 1
print(f"Number of connections to the same department: {department_connections}")
print(f"Number of connections of people with similar number of connections: {similar_connections}")

Number of connections to the same sector: 4471
Number of connections of people with similar number of connections: 171


##### Q2. Are all departments with 15 or more members more tightly connected amongst themselves in comparison to all individuals across the overall network irrespective of their department?  Where in this context, 'more tightly connected' is defined as having more mutual AND clustered connections. In addition to answering the overall question as yes or no, provide a list of departments this is true for (if any) and not true for (if any).

In [70]:
#CODE:
# Compute the reciprocity of the graph.
def computing_reciprocity(G):
    reciprocity_per_node = nx.reciprocity(G, G.nodes())
    reciprocity_values = []
    for value in reciprocity_per_node.values():
        if value is not None: reciprocity_values.append(float(value))
        else: reciprocity_values.append(0)
    mean_reciprocity = np.mean(reciprocity_values)
    return mean_reciprocity

# Create subgraphs
def create_subgraph(dep, dep_dict, G):
    # Select all members belonging to the department from the department dictionary.
    target_member_list = [member for member, m_dep in dep_dict.items() if m_dep == dep]
    # Select the nodes of these members from the graph G to create subgraphs.
    subgraph = G.subgraph(target_member_list).copy()
    return subgraph

# Find all departments with greater than or equal to 15 members, create a list of departments that qualify, and create a list of departments that do not qualify.
department_sizes = {}
# Calculate the number of members in each department.
for member, department in departments.items():
    if department not in department_sizes:
        department_sizes[department] = 0
    department_sizes[department] += 1
# Split large and small departments.
large_departments = {}
small_departments = {}
for dep, counts in department_sizes.items():
    if counts >= 15: large_departments[dep] = counts
    else: small_departments[dep] = counts

# Calculate the reciprocity and clustering coefficient for the overall network.
total_clustering = nx.average_clustering(eg)
total_mean_reciprocity = computing_reciprocity(eg)
# Initialise the clustering coefficients and average reciprocity for all large departments.
large_dep_mean_clustering = []
large_dep_mean_reciprocity = []
# More tightly connected departments list.
more_tightly_connected_deps = []
# Less tightly connected departments list.
less_tightly_connected_deps = []
# Compute the reciprocity and clustering coefficients within each department.
for department, size in large_departments.items():
    # Create a subgraph for the department.
    dep_subgraph = create_subgraph(department, departments, eg)
    # Calculate the average reciprocity and clustering coefficient for all nodes in this subgraph.
    this_dep_clustering = nx.average_clustering(dep_subgraph)
    this_reciprocity = computing_reciprocity(dep_subgraph)
    # Add results to the list.
    large_dep_mean_clustering.append(this_dep_clustering)
    large_dep_mean_reciprocity.append(this_reciprocity)
    if this_dep_clustering > total_clustering and this_reciprocity > total_mean_reciprocity:
        more_tightly_connected_deps.append(department)
    else: less_tightly_connected_deps.append(department)

# Calculate the average for all large departments.
mean_large_dep_clustering = np.mean(large_dep_mean_clustering)
mean_large_dep_reciprocity = np.mean(large_dep_mean_reciprocity)
# Compare to determine if there is greater connectivity within the department.
print(f"Average clustering coefficients of large departments: {mean_large_dep_clustering:.2f}")
print(f"Average reciprocity of large departments: {mean_large_dep_reciprocity:.2f}")
print(f"Average clustering coefficient of the totall graphs: {total_clustering:.2f}")
print(f"Average reciprocity of the total graph: {total_mean_reciprocity:.2f}")
print(f"More tightly connected departments: {more_tightly_connected_deps}")
print(f"Less tightly connected departments: {less_tightly_connected_deps}")

Average clustering coefficients of large departments: 0.51
Average reciprocity of large departments: 0.51
Average clustering coefficient of the totall graphs: 0.37
Average reciprocity of the total graph: 0.52
More tightly connected departments: ['14', '9', '17', '11', '10', '36', '8', '16', '13', '19']
Less tightly connected departments: ['1', '21', '7', '4', '22', '15', '6', '0', '23']
